In [ ]:
import numpy as np
import pandas as pd
import fastf1
import os
import warnings
import xlsxwriter
warnings.simplefilter(action='ignore', category=FutureWarning) #ignore future warnings so our file is not polluted with unnecessary info.

setting local cache folder - not needed, but execution is faster

In [ ]:
# Check if the subfolder exists
if not os.path.exists(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1"):
  # Create the subfolder
  os.makedirs(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1")

fastf1.Cache.enable_cache(os.getenv('LOCALAPPDATA') + "/pip/cache/fastF1")

In [ ]:
# Define the path of the subfolder
telemetries_path = os.getcwd() + "/Telemetries"

# Check if the subfolder exists
if not os.path.exists(telemetries_path):
  # Create the subfolder
  os.makedirs(telemetries_path)

In [ ]:
df_races = pd.DataFrame()
df_weather_data = pd.DataFrame()
df_races_control_data = pd.DataFrame()
df_drivers_info = pd.DataFrame()
df_telemetries = pd.DataFrame()
df_gears = pd.DataFrame()

Creating and storing dataframes following [1. Package exploration.ipynb](1.%20Package%20exploration.ipynb)

In [ ]:
for i in range(1,23):   #Events 1 to 22 from 22 Formula 1 World Competition.
    session = fastf1.get_session(2022, i, 'R')
    session.load()

    df_race = session.laps   #Dataframe with info from each lap from each driver from each session (Event)
    df_race['msLapTime'] = df_race['LapTime']//np.timedelta64(1, 'ms')   #Converts timedelta variable to float representing milliseconds
    df_race['EventName'] = session.event.EventName
    df_race['EventDate'] = session.event.EventDate
    df_races = pd.concat([df_races, df_race])

    session.weather_data['EventName'] = session.event.name   #Weather data for each event
    df_weather_data = pd.concat([df_weather_data,session.weather_data])

    session.race_control_messages['EventName'] = session.event.name  #Race control messages for each event
    df_races_control_data = pd.concat([df_races_control_data, session.race_control_messages])

    event_telemetry_path = os.path.join(telemetries_path, session.event.EventName + ".csv")
    telemetry_event = pd.DataFrame()

    for driver in session.drivers:

        df_driver_info = pd.DataFrame()    
        
        for item in list(session.get_driver(driver).index):
            
            df_driver_info['EventName'] = pd.Series(session.event.EventName)       
            df_driver_info[item] = session.get_driver(driver)[item]
        
        df_drivers_info = pd.concat([df_drivers_info,df_driver_info])

        session_d = session.laps.pick_driver(driver)
        
        for lap in range(len(session_d)): #len(session_d)
            print("Race " + str(i) + " of 22" + " - " "Driver " + str(session.drivers.index(driver)+1) + " of "+ str(len(session.drivers)) +" - " + "Lap " + str(lap + 1) +" of " + str(len(session_d)))
            telemetry_lap = session_d.iloc[lap,].get_telemetry() 
            telemetry_lap['DriverNumber'] = driver
            telemetry_lap['LapNumber'] = lap + 1
            telemetry_lap['EventName'] = session.event.EventName

            pd.concat([telemetry_lap,telemetry_event])
            
            gear_lap = telemetry_lap[['nGear','LapNumber','DriverNumber','EventName','RelativeDistance']]
            gear_lap = gear_lap.copy()
            gear_lap['RelativeDistance'] = gear_lap['RelativeDistance'].diff()
            gear_lap = gear_lap.groupby(['nGear','LapNumber','DriverNumber','EventName']).sum().reset_index()

            df_gears = pd.concat([df_gears, gear_lap])

telemetry_event.to_csv(event_telemetry_path)

df_gears = df_gears.pivot(index=['DriverNumber','LapNumber','EventName'], columns = ['nGear'])
df_gears = df_gears.rename(columns={'RelativeDistance': "Gear"})
df_gears.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_gears.columns]
df_gears.reset_index(inplace = True)

df_races = pd.merge(df_races,df_gears, how = 'left', on = ['DriverNumber','LapNumber','EventName'])

df_races.to_excel("./files/Races.xlsx")
df_weather_data.to_excel("./files/Weather.xlsx")
df_races_control_data.to_excel("./files/Control Messages.xlsx")
df_drivers_info.to_excel("./files/Driver Race Info.xlsx")


In [ ]:
from IPython.display import FileLink, FileLinks
FileLink('./1. Package exploration.ipynb')